# Using the eScriptorium Connector

This library provides a simple API to access the eScriptorium online platform.

Install with `pip install escriptorium-connector` and import it as follows.

In [5]:
from src.escriptorium_connector import EscriptoriumConnector
import os
from dotenv import load_dotenv
import io

## Instantiating the connector

It is probably a best practice to store your validation credentials separately. Here we use `python-dotenv` to load from a `.env` file the user's login credentials and the eScriptorium instance address.

The url of the eScriptorium instance is passed to the connector along with the url address for its api and the user's login credentials. The connector will take care of getting the API key for the user and any necessary cookies for asynchronous requests.

In [6]:
load_dotenv()
url = str(os.getenv('ESCRIPTORIUM_URL'))
api = f'{url}api/'
username = str(os.getenv('ESCRIPTORIUM_USERNAME'))
password = str(os.getenv('ESCRIPTORIUM_PASSWORD'))
escr = EscriptoriumConnector(url, api, username, password)

## Usage

Once instantiated, the connector provides well-documented and convenient methods to interact with your data on eScriptorium.

You might get a list of all your documents.

In [7]:
my_documents = escr.get_documents()
document_pk = my_documents.results[0].pk


In [8]:
print(len(my_documents.results))
print(my_documents.results[0])

12
GetDocument(pk=944, name='Segmentation_Complex_Undergrads', project='princeton_pgp_undergrad_project', main_script='Hebrew', read_direction=<ReadDirection.RTL: 'rtl'>, line_offset=<LineOffset.TOPLINE: 1>, parts_count=300, created_at=<DateTime '2021-10-16T21:23:35.285034Z' at 0x7f37fea7be20>, updated_at=<DateTime '2021-11-23T21:02:04.555713Z' at 0x7f37fea7bf40>, transcriptions=[GetTranscription(pk=1716, name='manual')], valid_block_types=[GetRegionType(pk=662, name='Arabic'), GetRegionType(pk=2, name='Main'), GetRegionType(pk=664, name='Oblique_135'), GetRegionType(pk=665, name='Oblique_225'), GetRegionType(pk=666, name='Oblique_315'), GetRegionType(pk=663, name='Oblique_45'), GetRegionType(pk=659, name='Upside_Down'), GetRegionType(pk=660, name='Vertical_Bottom_Up_90'), GetRegionType(pk=716, name='Vertical_Top_Down_270')], valid_line_types=[GetLineType(pk=3, name='Correction')], tags=[])


## Convenience

The connector provides several conveniences.  Perhaps the most broadly useful is automatically following the paging information to acquire the full dataset requested.  Here the request for document parts collects the full list of parts without the need to check if there are further pages of results available.

In [9]:
document_parts = escr.get_document_parts(document_pk)

In [10]:
print(len(document_parts))
selected_part = document_parts[1]
print(selected_part)

300
{'pk': 227518, 'name': '', 'filename': 'PGPID_5522_MS-TS-00012-00320-000-00001.jpg', 'title': 'Element 2', 'typology': None, 'image': {'uri': '/media/documents/944/PGPID_5522_MS-TS-00012-00320-000-00001.jpg', 'size': [1348, 2000], 'thumbnails': {'card': '/media/documents/944/PGPID_5522_MS-TS-00012-00320-000-00001.jpg.180x180_q85_crop-smart.jpg', 'large': '/media/documents/944/PGPID_5522_MS-TS-00012-00320-000-00001.jpg.1000x1000_q85.jpg'}}, 'image_file_size': 2693258, 'bw_image': None, 'workflow': {}, 'order': 1, 'recoverable': False, 'transcription_progress': 0, 'source': ''}


## Downloading Files

Images can be easily downloaded/uploaded with the connector. Also the exported XML files are available.

In [ ]:
image = escr.get_image(selected_part["image"]["uri"])

In [ ]:
from IPython.display import Image
display(Image(image))

In [ ]:
import zipfile

transcriptions = escr.get_document_transcriptions(document_pk)
page_xmls_zipped = escr.download_part_pagexml_transcription(document_pk, [x["pk"] for x in document_parts[0:4]], transcriptions[0]["pk"])

# the bytes from the connector are from a zip file, so we need to unzip it
zip = zipfile.ZipFile(io.BytesIO(page_xmls_zipped))
page_xmls = [zip.read(x) for x in zip.infolist()]
print(len(page_xmls))

In [ ]:
import xml.dom.minidom as md

# Let's see what it looks like:
for page_xml in page_xmls:
    dom = md.parse(io.BytesIO(page_xml))
    pretty_xml = dom.toprettyxml()
    # remove the weird newline issue:
    pretty_xml = os.linesep.join([s for s in pretty_xml.splitlines() if s.strip()])
    print(pretty_xml)